In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!ls '/content/drive/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
'My Drive'


In [ ]:
import os
# Create a new directory 'Tweets' to store the crawled tweets
path = '/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets'
os.mkdir(path)

In [ ]:
# download libraries for tweets scraping
!pip install GetOldTweets3
!pip install unicodecsv

In [ ]:
# download libraries for tweets analysis
!pip install -U textblob-de

Requirement already up-to-date: textblob-de in /usr/local/lib/python3.6/dist-packages (0.4.3)


**Part 1: Tweets scraping**

In [ ]:
# import libraries
import tweepy
import GetOldTweets3 as got
import unicodecsv as csv

In [ ]:
# input the user access tokens
consumer_key = '4ATVIJAX4vvGK5FumwjP47puP'
consumer_secret = 'cfdOnzB53poxC3T6N4RUCBhiyBD3zjomAjkjJYPV7BGAKoHSE0'

access_token = '1079131550179037186-JRa2cfOwLcl84VGTjUUi30mUtJRyHz'
access_token_secret = 'NcK1W6DG3emIihoYd1PZ28qIu4O06E7aTh84NdkBinzXG'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# For year 2015 to 2020 crawl tweets with the setted criterion and write it in csv files
# download it in the folder 'Tweets'
for Year in range(2015, 2021):
    i = 1
    while i in range(1, 13):
        if i in range(1, 12):
            tweetCriteria = got.manager.TweetCriteria().setQuerySearch('Bier' or 'Brauerei')\
                                                       .setSince("{}-{}-01".format(Year, i))\
                                                       .setUntil("{}-{}-01".format(Year, i + 1))\
                                                       .setMaxTweets(0)\
                                                       .setLang('de')\
                                                       .setNear('Hessen, Germany')\
                                                       .setWithin('400km')
            tweet = got.manager.TweetManager.getTweets(tweetCriteria)
            # Open/create a file to append data to
            csvFile = open('/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets/result_{}_{}.csv'.format(Year, i), 'wb+')
            #Use csv writer
            csvWriter = csv.writer(csvFile)

            for t in tweet:
                # Write a row to the CSV file.
                csvWriter.writerow([t.formatted_date, t.text])
                #print(t.formatted_date, t.text)
            csvFile.close()
            
        else:
            tweetCriteria = got.manager.TweetCriteria().setQuerySearch('Bier' or 'Brauerei')\
                                                       .setSince("{}-12-01".format(Year))\
                                                       .setUntil("{}-1-01".format(Year + 1))\
                                                       .setMaxTweets(0)\
                                                       .setLang('de')\
                                                       .setNear('Hessen, Germany')\
                                                       .setWithin('400km')
            tweet = got.manager.TweetManager.getTweets(tweetCriteria)
            # Open/create a file to append data to
            csvFile = open('/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets/result_{}_12.csv'.format(Year), 'wb+')
            #Use csv writer
            csvWriter = csv.writer(csvFile)

            for t in tweet:
                # Write a row to the CSV file.
                csvWriter.writerow([t.formatted_date, t.text])
                #print(t.formatted_date, t.text)
            csvFile.close()
            
        i = i + 1

An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=Bier%20near%3A%22Hessen%2C%20Germany%22%20within%3A400km%20since%3A2020-7-01%20until%3A2020-8-01&src=typd


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Part 2: Tweets sentiment analysis**

In [ ]:
# import the libraries
from textblob_de import TextBlobDE as TextBlob
import pandas as pd
import re
import csv

In [ ]:
# clean tweets with regular expression
FLAGS = re.MULTILINE | re.DOTALL
def cleanTxt(text):
    text = re.sub(r"@\S+", "<user>", text, flags=FLAGS) #replace @mentions with '<user>'
    text = re.sub(r'#', '', text, flags=FLAGS) #remove the '#' symbol 
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*|~http", "<url>", text, flags=FLAGS) #replace the hyperlink with '<url>'
    return text

In [ ]:
# create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
#create a function to compute the negative, neutral and postive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Open/create a file to append data to
csvFile = open('/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets/Hessen_2015_2020_percentages_of_pntweets.csv', 'w')

#Use csv writer
csvWriter = csv.writer(csvFile)

csvWriter.writerow(["Year_Month","Tweet_Counts","Positive_Percentage(%)", "Negative_Percentage(%)"])


# data of the year 2015 to 2019 from month 1 to 12
for year in range(2015, 2020):
    for i in range(1, 13):
        with open('/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets/result_{}_{}.csv'.format(year, i)) as csvfile:

            header_list = ["Datetime", "Tweet"]
            log = pd.read_csv(csvfile, header=None, names=header_list) 
            #'Bier' or 'Brauerei' are in the sentence
            df = pd.DataFrame([tweet for tweet in log['Tweet'] if 'Bier' or 'Brauerei' in tweet], columns=['Tweets'])
            # clean the text
            df['Tweets'] = df['Tweets'].apply(cleanTxt)
            # create new columns
            df['Polarity'] = df['Tweets'].apply(getPolarity)
            df['Analysis'] = df['Polarity'].apply(getAnalysis)

            # get the percentage of positive tweets
            ptweets = df[df.Analysis == 'Positive']
            ptweets = ptweets['Tweets']

            per_ptweets = round((ptweets.shape[0]/df.shape[0])*100, 1)

            # get the percentage of negative tweets
            ntweets = df[df.Analysis == 'Negative']
            ntweets = ntweets['Tweets']

            per_ntweets = round((ntweets.shape[0]/df.shape[0])*100, 1)

            # Write a row to the CSV file.
            csvWriter.writerow(['{}_{}'.format(year, i),df.shape[0],per_ptweets,per_ntweets])


# data of the year 2020 from month 1 to 6
for i in range(1, 7):
        with open('/content/drive/My Drive/AI Capstone/Tweets month data from 2015 to 2020/Tweets/result_2020_{}.csv'.format(i)) as csvfile:

            header_list = ["Datetime", "Tweet"]
            log = pd.read_csv(csvfile, header=None, names=header_list) 
            #'Bier' or 'Brauerei' are in the sentence
            df = pd.DataFrame([tweet for tweet in log['Tweet'] if 'Bier' or 'Brauerei' in tweet], columns=['Tweets'])
            # clean the text
            df['Tweets'] = df['Tweets'].apply(cleanTxt)
            # create new columns
            df['Polarity'] = df['Tweets'].apply(getPolarity)
            df['Analysis'] = df['Polarity'].apply(getAnalysis)

            # get the percentage of positive tweets
            ptweets = df[df.Analysis == 'Positive']
            ptweets = ptweets['Tweets']

            per_ptweets = round((ptweets.shape[0]/df.shape[0])*100, 1)

            # get the percentage of negative tweets
            ntweets = df[df.Analysis == 'Negative']
            ntweets = ntweets['Tweets']

            per_ntweets = round((ntweets.shape[0]/df.shape[0])*100, 1)

            # Write a row to the CSV file.
            csvWriter.writerow(['{}_{}'.format(year, i), df.shape[0],per_ptweets,per_ntweets])

csvFile.close() 